This is a demo for a complete autoemulate pipeline 

you have a simulation , put your simulator in a simulator object 


In [10]:
from autoemulate.simulations.naghavi_cardiac_ModularCirc import extract_parameter_ranges
# Usage example:
parameters_range = extract_parameter_ranges('/Users/mfamili/work/ModularCirc/Tutorials/Tutorial_03/Parameters_01.json')
parameters_range

{'ao.r': (120.0, 360.0),
 'ao.c': (0.15, 0.44999999999999996),
 'art.r': (562.5, 1687.5),
 'art.c': (1.5, 4.5),
 'ven.r': (4.5, 13.5),
 'ven.c': (66.65, 199.95000000000002),
 'av.r': (3.0, 9.0),
 'mv.r': (2.05, 6.1499999999999995),
 'la.E_pas': (0.22, 0.66),
 'la.E_act': (0.225, 0.675),
 'la.v_ref': (5.0, 15.0),
 'la.k_pas': (0.01665, 0.07500000000000001),
 'lv.E_pas': (0.5, 1.5),
 'lv.E_act': (1.5, 4.5),
 'lv.v_ref': (5.0, 15.0),
 'lv.k_pas': (0.00999, 0.045)}

In [11]:
import pandas as pd
import numpy as np
from autoemulate.experimental_design import LatinHypercube


# Generate Latin Hypercube samples
N_samples = 60
lhd = LatinHypercube(list(parameters_range.values()))
sample_array = lhd.sample(N_samples)
sample_df = pd.DataFrame(sample_array, columns=parameters_range.keys())

print("Number of parameters:", sample_df.shape[1], "Number of samples from each parameter:", sample_df.shape[0])
sample_df.head()

Number of parameters: 16 Number of samples from each parameter: 60


,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,la.v_ref,la.k_pas,lv.E_pas,lv.E_act,lv.v_ref,lv.k_pas
0,334.247200,0.298126,1350.339838,1.878151,12.089577,110.041047,8.570339,4.966271,0.381802,0.581709,5.187959,0.034208,1.090765,2.117194,12.542671,0.015339
1,317.081500,0.392056,1385.423091,2.726083,8.005873,186.744788,6.715292,4.115116,0.524411,0.587504,11.651372,0.042936,1.280037,3.137457,13.798822,0.010986
2,188.924960,0.231354,1517.702080,3.955581,5.599686,119.413594,4.013578,5.303890,0.263311,0.475333,5.095720,0.069382,1.379846,1.776955,13.903043,0.033848
3,343.042923,0.399963,1284.803060,3.559930,11.055576,151.606996,3.329687,3.337612,0.243984,0.465899,12.162585,0.058591,0.783127,2.911347,14.962053,0.022136
4,248.324563,0.165977,919.434490,2.253556,10.583195,101.906029,8.477522,3.946127,0.226862,0.321755,8.056464,0.020150,0.638783,3.388206,12.176559,0.030031


In [12]:
from autoemulate.simulations.naghavi_cardiac_ModularCirc import NaghaviSimulator

# Initialize the simulator
simulator = NaghaviSimulator(n_cycles=1000, dt=0.001)

# Run batch simulations with the samples generated in Cell 1
results = simulator.run_batch_simulations(sample_df)

# Convert results to DataFrame for analysis
results_df = pd.DataFrame(results)


Running simulations:   0%|          | 0/60 [00:00<?, ?sample/s]

Successfully completed 60/60 simulations (100.0%)


In [13]:
results_df.describe()

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,la.v_ref,la.k_pas,lv.E_pas,lv.E_act,lv.v_ref,lv.k_pas
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,240.144601,0.300008,1124.802745,2.998736,8.998322,133.353206,5.997795,4.098871,0.440425,0.450209,9.995860,0.045818,0.999738,3.001080,10.001049,0.027489
std,69.864370,0.087405,328.080066,0.872015,2.625963,38.766292,1.753916,1.189523,0.128150,0.130899,2.927277,0.016978,0.290426,0.875289,2.916978,0.010171
min,120.851584,0.152636,578.657920,1.519576,4.542914,68.797237,3.097075,2.115809,0.226862,0.226365,5.095720,0.017442,0.515731,1.549122,5.049720,0.010389
25%,182.503622,0.224668,853.064929,2.244686,6.714384,101.286102,4.514276,3.113774,0.329338,0.340839,7.555261,0.031070,0.748673,2.256165,7.576440,0.018979
50%,240.558239,0.300678,1121.138657,2.999078,8.983392,133.480469,6.041953,4.076417,0.439533,0.448313,9.973850,0.045784,0.994735,3.001396,10.018977,0.027471
75%,299.482780,0.373220,1407.853868,3.736246,11.242930,166.890197,7.496350,5.088525,0.550731,0.561519,12.442595,0.060322,1.252377,3.741546,12.413601,0.036154
max,358.412888,0.446948,1675.143810,4.477666,13.386768,197.872961,8.929164,6.109173,0.656608,0.671993,14.999613,0.074161,1.493644,4.481787,14.962053,0.044979


test your simulator with our test function to make sure it ios compatible wih AutoEmulate pipelien 

In [14]:
# need a test for the simulator 

In [19]:
results_df.head()

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,la.P_o,la.Q_i,la.Q_o,la.V,lv.P,lv.P_i,lv.P_o,lv.Q_i,lv.Q_o,lv.V
0,334.247200,0.298126,1350.339838,1.878151,12.089577,110.041047,8.570339,4.966271,0.381802,0.581709,...,"[40.04067347593265, 40.036089392716015, 40.031...","[-2.575050406038219, -2.574669242288889, -2.57...","[7.388980352594501, 7.387956074297883, 7.38693...","[93.6, 93.59003737462166, 93.58007613600397, 9...","[3.344995295156757, 3.3454980554485165, 3.3460...","[3.344995295156757, 3.3454980554485165, 3.3460...","[3.344995295156757, 3.3454980554485165, 3.3460...","[7.388980352594501, 7.387956074297883, 7.38693...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[104.0, 104.00738795611932, 104.0147749015686,..."
1,317.081500,0.392056,1385.423091,2.726083,8.005873,186.744788,6.715292,4.115116,0.524411,0.587504,...,"[40.118114617784876, 40.10982010907997, 40.101...","[-4.355323719223735, -4.354284722311319, -4.35...","[9.222103049352121, 9.220002533539748, 9.21790...","[93.6, 93.5864257115181, 93.57285444090711, 93...","[2.1680888544500263, 2.168438212417316, 2.1687...","[2.1680888544500263, 2.168438212417316, 2.1687...","[2.1680888544500263, 2.168438212417316, 2.1687...","[9.222103049352121, 9.220002533539748, 9.21790...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[104.00000000000001, 104.00922000335386, 104.0..."
2,188.924960,0.231354,1517.702080,3.955581,5.599686,119.413594,4.013578,5.303890,0.263311,0.475333,...,"[62.78053779067132, 62.738189377720246, 62.695...","[-9.745263184387019, -9.737685941583159, -9.73...","[6.605536176122244, 6.59632537653942, 6.587127...","[93.6, 93.58366551874924, 93.56734358961644, 9...","[27.745500516117207, 27.752005170972975, 27.75...","[27.745500516117207, 27.752005170972975, 27.75...","[27.745500516117207, 27.752005170972975, 27.75...","[6.605536176122244, 6.59632537653942, 6.587127...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[104.0, 104.00659658321501, 104.01318627931053..."
3,343.042923,0.399963,1284.803060,3.559930,11.055576,151.606996,3.329687,3.337612,0.243984,0.465899,...,"[35.51295758059091, 35.50380144202827, 35.4946...","[-2.627293086584822, -2.626463305605376, -2.62...","[9.190775239980267, 9.18768932610401, 9.184605...","[93.6, 93.5881858438275, 93.57637537596842, 93...","[4.837712590934286, 4.83885603665039, 4.839999...","[4.837712590934286, 4.83885603665039, 4.839999...","[4.837712590934286, 4.83885603665039, 4.839999...","[9.190775239980267, 9.18768932610401, 9.184605...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[104.0, 104.00918769211658, 104.01837247747224..."
4,248.324563,0.165977,919.434490,2.253556,10.583195,101.906029,8.477522,3.946127,0.226862,0.321755,...,"[20.66270105780273, 20.66199100804784, 20.6612...","[-1.0433590819294625, -1.043290946159945, -1.0...","[2.8466474216865785, 2.846249363170714, 2.8458...","[93.6, 93.5961104596853, 93.59222138239498, 93...","[9.429469391344384, 9.430330130964895, 9.43119...","[9.429469391344384, 9.430330130964895, 9.43119...","[9.429469391344384, 9.430330130964895, 9.43119...","[2.8466474216865785, 2.846249363170714, 2.8458...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[103.99999999999999, 104.00284624936774, 104.0..."


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from ModularCirc.Models.NaghaviModel import NaghaviModel, NaghaviModelParameters
from ModularCirc.Analysis.BaseAnalysis import BaseAnalysis
from ModularCirc.Solver import Solver
import pandas as pd
import os
import joblib
from tqdm import tqdm
from typing import Dict, Tuple, List
from autoemulate.history_matching import HistoryMatching


class NaghaviHistoryMatcher:
    def __init__(self, observed_data: Dict[str, np.ndarray], 
                param_ranges: Dict[str, Tuple[float, float]],
                n_cycles: int = 40, dt: float = 0.001,
                implausibility_threshold: float = 3.0,
                model_discrepancy: float = 0.0):
        """
        Initialize the history matching framework for Naghavi model
        
        Args:
            observed_data: Dictionary containing observed signals (keys: signal names, values: arrays)
            param_ranges: Dictionary of parameter ranges to explore (keys: parameter names, values: (min, max))
            n_cycles: Number of cardiac cycles to simulate
            dt: Time step for simulation
            implausibility_threshold: Threshold for considering points implausible
            model_discrepancy: Model discrepancy term
        """
        self.observed_data = observed_data
        self.param_ranges = param_ranges
        self.param_names = list(param_ranges.keys())
        self.n_cycles = n_cycles
        self.dt = dt
        self.samples = None
        self.simulator_outputs = None
        self.gp_emulators = {}
        
        # Initialize the HistoryMatching object
        self.history_matcher = HistoryMatching(
            threshold=implausibility_threshold,
            discrepancy=model_discrepancy,
            rank=1
        )
        
        # Time setup dictionary
        self.time_setup = {
            "name": "HistoryMatching",
            "ncycles": n_cycles,
            "tcycle": 1.0,  # Will be adjusted per simulation
            "dt": dt,
            "export_min": 1
        }
        
        
    def signal_get_pulse(self, signal: np.ndarray, dt: float, num: int = 100) -> Tuple[float, np.ndarray]:
        """
        Resample raw signal to standard resolution
        
        Args:
            signal: Raw signal array
            dt: Original time resolution
            num: Number of points in resampled signal
            
        Returns:
            Tuple of (new_dt, resampled_signal)
        """
        ind = np.argmin(signal)        
        ncycle = len(signal)
        new_signal = np.interp(np.linspace(0, ncycle, num), 
                            np.arange(ncycle), 
                            np.roll(signal, -ind))
        new_dt = ncycle / (num - 1) * dt
        return new_dt, new_signal
    
    def run_simulation(self, params: Dict[str, float]) -> Dict[str, float]:
        """
        Run a single Naghavi model simulation with given parameters
        
        Args:
            params: Dictionary of parameter values
            
        Returns:
            Dictionary of processed features from simulation outputs
        """
        # Create parameter object
        parobj = NaghaviModelParameters()
        
        # Set parameters from input
        for param_name, value in params.items():
            if param_name == "T":
                continue  # Handle cycle time separately
            try:
                obj, param = param_name.split('.')
                parobj._set_comp(obj, [obj], **{param: value})
            except Exception as e:
                print(f"Error setting parameter {param_name}: {e}")
                return None
        
        # Adjust cycle time if specified
        t_cycle = params.get("T", 1.0)
        self.time_setup["tcycle"] = t_cycle
        
        # Create and run model
        try:
            model = NaghaviModel(time_setup_dict=self.time_setup, parobj=parobj, suppress_printing=True)
            solver = Solver(model=model)
            solver.setup(suppress_output=True, optimize_secondary_sv=False, conv_cols=["p_ao"], method='LSODA')
            solver.solve()
            
            if not solver.converged:
                return None
        except Exception as e:
            print(f"Simulation error: {e}")
            return None
        
        # Process results
        raw_results = {}
        
        # Get indices for the last cycle
        tind_fin = np.arange(start=model.time_object.n_t-model.time_object.n_c,
                            stop=model.time_object.n_t)
        
        # Extract raw signals of interest
        for signal_name in self.observed_data.keys():
            if signal_name == "CO":
                # Handle cardiac output separately
                continue
                
            try:
                component, _ = signal_name.split('.')
                # Get the pressure signal from the component
                if component in model.components:
                    # Extract P_i (pressure) from the component for the last cycle
                    raw_signal = model.components[component].P_i.values[tind_fin]
                    
                    if '_min' in signal_name:
                        raw_results[signal_name] = np.min(raw_signal)
                    elif '_max' in signal_name:
                        raw_results[signal_name] = np.max(raw_signal)
                    elif '_pc' in signal_name:
                        # For PCA components, you would need to implement the PCA calculation
                        # This is a placeholder
                        pc_index = int(signal_name.split('_pc')[1])
                        # Simple approximation for demonstration
                        raw_results[signal_name] = np.mean(raw_signal) * (pc_index * 0.1)
                else:
                    print(f"Component {component} not found in model")
                    return None
            except Exception as e:
                print(f"Error extracting signal {signal_name}: {e}")
                return None
                
        # Add scalar metrics
        analysis = BaseAnalysis(model)
        analysis.compute_cardiac_output("lv")
        raw_results["CO"] = analysis.CO
        
        return raw_results  # Return the processed results directly

    def process_simulation_output(self, raw_results: Dict[str, np.ndarray]) -> Dict[str, float]:
        """
        Process raw simulation outputs to extract features for comparison with observed data
        
        Args:
            raw_results: Dictionary of raw simulation outputs
            
        Returns:
            Dictionary of processed features
        """
        processed = {}
        
        # Process each signal based on what features we want to extract
        for signal_name, signal_data in raw_results.items():
            if signal_name == "CO":
                # Cardiac output is already a scalar
                processed[signal_name] = signal_data
            elif '.P' in signal_name:
                # For pressure signals, extract min, max and other features
                processed[f"{signal_name}_min"] = np.min(signal_data)
                processed[f"{signal_name}_max"] = np.max(signal_data)
                
                # Could add more sophisticated feature extraction here
                # For example, PCA components if needed
            
        return processed
    
    def run_wave(self, samples: pd.DataFrame, n_jobs: int = 1) -> Tuple[pd.DataFrame, List[Dict], np.ndarray]:
        """
        Run a wave of simulations
        
        Args:
            samples: DataFrame of parameter samples
            n_jobs: Number of parallel jobs
            
        Returns:
            Tuple of (successful_samples, outputs, implausibility_scores)
        """
        successful_samples = []
        simulation_outputs = []
        implausibility_scores = []

        # Serial execution
        for _, row in tqdm(samples.iterrows(), total=len(samples)):
            result = self._run_single_sample(row)
            if result is not None:
                params, outputs, impl = result
                successful_samples.append(params)
                simulation_outputs.append(outputs)
                implausibility_scores.append(impl)
                        
        # Create a dataframe with all outputs for inspection
        if successful_samples:
            # Combine parameters and outputs into one dataframe for inspection
            output_df = pd.DataFrame(successful_samples)
            
            # Add implausibility scores
            output_df['implausibility'] = implausibility_scores
            
            # Add simulation outputs for each observed signal
            for i, output_dict in enumerate(simulation_outputs):
                for signal_name, value in output_dict.items():
                    if signal_name not in output_df.columns:
                        output_df[signal_name] = np.nan
                    output_df.at[i, signal_name] = value
            
            # Print summary statistics of the outputs
            print("\nOutput DataFrame Summary:")
            print(output_df.describe())
            
            # Show the relationship between parameters and implausibility
            print("\nCorrelation between parameters and implausibility:")
            for param in self.param_names:
                corr = np.corrcoef(output_df[param], output_df['implausibility'])[0, 1]
                print(f"{param}: {corr:.4f}")
            
            # Show the best match in this wave
            best_idx = np.argmin(implausibility_scores)
            print("\nBest match in this wave:")
            print(output_df.iloc[[best_idx]])
            
            # Print the observed vs. simulated values for the best match
            print("\nObserved vs. Best Simulated Values:")
            for signal_name, observed_value in self.observed_data.items():
                if signal_name in output_df.columns:
                    simulated = output_df.iloc[best_idx][signal_name]
                    print(f"{signal_name}: Observed={observed_value:.4f}, Simulated={simulated:.4f}, Difference={abs(observed_value - simulated):.4f}")
            
            # Save the output dataframe to a CSV file for external analysis
            output_df.to_csv(f"wave_outputs.csv", index=False)
            print("\nOutput dataframe saved to 'wave_outputs.csv' for further analysis")
            
        return pd.DataFrame(successful_samples), simulation_outputs, np.array(implausibility_scores)
    def _run_single_sample(self, sample: pd.Series) -> Tuple[Dict, Dict, float]:
        """
        Run a single sample and calculate implausibility using HistoryMatching class
        
        Args:
            sample: Parameter values as Series
            
        Returns:
            Tuple of (parameters, outputs, implausibility) or None if failed
        """
        params = sample.to_dict()
        outputs = self.run_simulation(params)
        
        if outputs is None:
            return None
            
        # Calculate implausibility using the HistoryMatching class
        implausibilities = []
        
        for signal_name, obs_value in self.observed_data.items():
            if signal_name not in outputs:
                print(f"Warning: {signal_name} not found in simulation outputs")
                continue
                
            # Set up observation and prediction in format expected by HistoryMatching
            # [mean, variance]
            obs = [obs_value, 1.0]  # Assuming observation variance of 1.0
            pred = [outputs[signal_name], 0.1]  # Assuming prediction variance of 0.1
            
            try:
                result = self.history_matcher.history_matching(obs, pred)
                implausibilities.append(result["I"][0])  # Take the first implausibility value
            except Exception as e:
                print(f"Error calculating implausibility for {signal_name}: {e}")
                implausibilities.append(float('inf'))  # Treat as infinitely implausible
        
        if not implausibilities:
            return None  # No valid implausibility calculations
            
        impl = np.max(implausibilities)  # Maximum implausibility approach
        
        return params, outputs, impl
    def history_matching(self, n_waves: int = 3, n_samples_per_wave: int = 100, n_jobs: int = 4):
        """
        Perform iterative history matching using the HistoryMatching class
        
        Args:
            n_waves: Number of waves to perform
            n_samples_per_wave: Samples per wave
            n_jobs: Number of parallel jobs
        """
        current_samples = self.generate_samples(n_samples_per_wave)
        all_samples = pd.DataFrame()
        all_implausibilities = np.array([])
        
        for wave in range(n_waves):
            print(f"\nStarting wave {wave + 1}/{n_waves}")
            
            # Run simulations
            successful_samples, outputs, impl_scores = self.run_wave(current_samples, n_jobs)
            print(f"Wave {wave+1}: {len(successful_samples)} successful samples out of {len(current_samples)}")
            
            if len(successful_samples) == 0:
                print("No successful simulations in this wave, generating new random samples")
                current_samples = self.generate_samples(n_samples_per_wave)
                continue  # Try again with new samples instead of stopping
                
            # Store all results
            all_samples = pd.concat([all_samples, successful_samples])
            all_implausibilities = np.concatenate([all_implausibilities, impl_scores]) if len(all_implausibilities) > 0 else impl_scores
            
            # Identify not implausible points (NROY) using threshold from HistoryMatching
            nroy_indices = np.where(impl_scores <= self.history_matcher.threshold)[0]
            not_implausible = successful_samples.iloc[nroy_indices] if len(nroy_indices) > 0 else pd.DataFrame()
            
            print(f"Wave {wave + 1}: {len(not_implausible)} not implausible points found (threshold: {self.history_matcher.threshold})")
            
            # Prepare next wave samples
            if wave < n_waves - 1:
                if len(not_implausible) >= 2:  # Need at least 2 points for meaningful bounds
                    try:
                        print("Generating new samples based on not implausible points")
                        new_sample_points = self.history_matcher._sample_new_points(
                            not_implausible[self.param_names].values, n_samples_per_wave)
                        current_samples = pd.DataFrame(new_sample_points, columns=self.param_names)
                    except Exception as e:
                        print(f"Error generating new samples: {e}")
                        print("Falling back to random sampling")
                        current_samples = self.generate_samples(n_samples_per_wave)
                else:
                    # Fall back to random sampling if too few NROY points
                    print("Too few not implausible points, generating new random samples")
                    current_samples = self.generate_samples(n_samples_per_wave)
                
                print(f"Generated {len(current_samples)} new samples for wave {wave+2}")
            
            # Plot results only if we have implausible and not implausible points
            if len(not_implausible) > 0 and len(successful_samples) > len(not_implausible):
                self.plot_wave_results(wave, successful_samples, impl_scores, not_implausible)
        
        print(f"\nHistory matching completed. Total samples: {len(all_samples)}")
        
        return all_samples, all_implausibilities
    
    def generate_samples(self, n_samples: int, method: str = 'random') -> pd.DataFrame:
        """
        Generate parameter samples using specified method
        
        Args:
            n_samples: Number of samples to generate
            method: Sampling method ('random' or 'lhs')
            
        Returns:
            DataFrame of parameter samples
        """
        samples = []
        
        if method == 'random':
            for param_name, (low, high) in self.param_ranges.items():
                samples.append(np.random.uniform(low, high, n_samples))
        elif method == 'lhs':
            # Implement Latin Hypercube Sampling here if needed
            pass
        else:
            raise ValueError(f"Unknown sampling method: {method}")
            
        return pd.DataFrame(np.array(samples).T, columns=self.param_names)
    
    def plot_wave_results(self, wave: int, samples: pd.DataFrame, 
                        impl_scores: np.ndarray, not_implausible: pd.DataFrame):
        """
        Plot results from a history matching wave
        
        Args:
            wave: Wave number
            samples: All successful samples
            impl_scores: Corresponding implausibility scores
            not_implausible: Not implausible samples
        """
        if len(self.param_names) < 2:
            return
            
        # Select two main parameters to plot
        param1, param2 = self.param_names[:2]
        
        plt.figure(figsize=(12, 5))
        
        # Plot all samples colored by implausibility
        plt.subplot(1, 2, 1)
        sc = plt.scatter(samples[param1], samples[param2], c=impl_scores, 
                        cmap='viridis_r', vmax=3.0)
        plt.colorbar(sc, label='Implausibility')
        plt.xlabel(param1)
        plt.ylabel(param2)
        plt.title(f'Wave {wave + 1} - All Samples')
        
        # Plot not implausible region
        plt.subplot(1, 2, 2)
        plt.scatter(not_implausible[param1], not_implausible[param2], 
                c='blue', alpha=0.5)
        plt.xlabel(param1)
        plt.ylabel(param2)
        plt.title(f'Wave {wave + 1} - Not Implausible Region')
        
        plt.tight_layout()
        plt.show()

    def calculate_implausibility(self, sim_output: Dict[str, float], 
                            obs_error: Dict[str, float], 
                            model_error: Dict[str, float],
                            emulator_error: Dict[str, float] = None) -> float:
        """
        Calculate combined implausibility metric across all outputs
        
        Args:
            sim_output: Simulator outputs (max values)
            obs_error: Observation errors for each output
            model_error: Model errors for each output
            emulator_error: Optional emulator errors
            
        Returns:
            Combined implausibility score
        """
        total_implausibility = 0.0
        
        for signal_name, obs_value in self.observed_data.items():
            sim_value = sim_output[signal_name]
            error = obs_error.get(signal_name, 0.1)
            m_error = model_error.get(signal_name, 0.05)
            
            # Add emulator error if provided
            e_error = emulator_error.get(signal_name, 0.0) if emulator_error else 0.0
            
            # Calculate absolute difference
            diff = np.abs(sim_value - obs_value)
            impl = diff / np.sqrt(error**2 + m_error**2 + e_error**2)
            total_implausibility += impl
                
        return total_implausibility

    def calculate_implausibility(self, sim_output: Dict[str, float], 
                                obs_error: Dict[str, float], 
                                model_error: Dict[str, float],
                                emulator_error: Dict[str, float] = None) -> float:
        """
        Calculate combined implausibility metric across all processed outputs
        
        Args:
            sim_output: Simulator processed outputs
            obs_error: Observation errors for each output
            model_error: Model errors for each output
            emulator_error: Optional emulator errors
            
        Returns:
            Combined implausibility score
        """
        implausibilities = []
        
        for signal_name, obs_value in self.observed_data.items():
            if signal_name not in sim_output:
                print(f"Warning: {signal_name} not in simulation output")
                continue
                
            sim_value = sim_output[signal_name]
            error = obs_error.get(signal_name, 0.1)
            m_error = model_error.get(signal_name, 0.05)
            
            # Add emulator error if provided
            e_error = emulator_error.get(signal_name, 0.0) if emulator_error else 0.0
            
            # Calculate absolute difference
            diff = np.abs(sim_value - obs_value)
            impl = diff / np.sqrt(error**2 + m_error**2 + e_error**2)
            implausibilities.append(impl)
                
        # Return maximum implausibility (conservative approach)
        # Alternative: return np.mean(implausibilities) for average implausibility
        return np.max(implausibilities)

# Example usage
if __name__ == "__main__":
    # Define observed data (would normally load from file)

    observed_signals = {
        "ao.P_min": 80.0,    # Minimum observed aortic pressure (mmHg)
        "ao.P_max": 120.0,   # Maximum observed aortic pressure (mmHg)
        "ao.P_pc1": 10.5,    # First PCA component of aortic pressure
        "ao.P_pc2": -3.2,    # Second PCA component of aortic pressure
        "ao.P_pc3": 0.8,     # Third PCA component of aortic pressure
        "CO": 5.0            # Observed cardiac output in L/min
    }


# Updated parameter ranges
    param_ranges = {
        "lv.E_act": (0.0, 20.0),  # LV active elastance (similar to end-systolic elastance)
        "lv.v_ref": (0.0, 20.0),  # LV reference volume (similar to dead volume)
        "la.E_act": (0., 10.5),   # LA active elastance
        # For systemic arterial resistance, we need to check what parameter name to use
    }

    
    # Create history matcher instance
    hm = NaghaviHistoryMatcher(observed_data=observed_signals, param_ranges=param_ranges)
    
    # Run history matching
    best_samples, best_impl_scores = hm.history_matching(n_waves=5, n_samples_per_wave=50)
    
    # Print best match
    best_idx = np.argmin(best_impl_scores)
    print("\nBest matching parameters:")
    print(best_samples.iloc[best_idx])
    print(f"Implausibility: {best_impl_scores[best_idx]}")


Starting wave 1/5


100%|██████████| 50/50 [00:02<00:00, 19.73it/s]



Output DataFrame Summary:
        lv.E_act   lv.v_ref   la.E_act  implausibility      ao.P_min  \
count  50.000000  50.000000  50.000000       50.000000  5.000000e+01   
mean   10.338103   9.105457   4.648469       85.233255  9.853282e+01   
std     5.850657   5.166678   2.867269       84.201274  9.246473e-07   
min     0.372654   0.486470   0.014313       27.490180  9.853282e+01   
25%     5.439365   5.505518   2.302249       27.490181  9.853282e+01   
50%     9.728371   8.609048   4.754798       27.490181  9.853282e+01   
75%    16.707942  12.960002   6.919309      130.460271  9.853282e+01   
max    19.165977  19.632563  10.459712      292.905097  9.853283e+01   

           ao.P_max      ao.P_pc1      ao.P_pc2      ao.P_pc3          CO  
count  5.000000e+01  5.000000e+01  5.000000e+01  5.000000e+01   50.000000  
mean   9.901466e+01  9.877315e+00  1.975463e+01  2.963194e+01   76.594467  
std    3.834653e-07  6.398937e-08  1.279787e-07  1.919681e-07  101.305455  
min    9.901466e+01 

100%|██████████| 50/50 [00:02<00:00, 21.36it/s]



Output DataFrame Summary:
        lv.E_act   lv.v_ref   la.E_act  implausibility      ao.P_min  \
count  50.000000  50.000000  50.000000       50.000000  5.000000e+01   
mean    8.824617  10.447820   5.026994       91.390765  9.853282e+01   
std     5.439802   5.895540   2.793180       87.545319  7.069771e-07   
min     0.035987   0.408410   0.325296       27.490181  9.853282e+01   
25%     4.079524   5.584872   2.787769       27.490181  9.853282e+01   
50%     9.376542  10.174185   4.883951       27.490181  9.853282e+01   
75%    12.190645  15.709998   7.465198      156.575878  9.853282e+01   
max    19.298929  19.914855  10.366954      312.654873  9.853283e+01   

           ao.P_max      ao.P_pc1      ao.P_pc2      ao.P_pc3          CO  
count  5.000000e+01  5.000000e+01  5.000000e+01  5.000000e+01   50.000000  
mean   9.901466e+01  9.877315e+00  1.975463e+01  2.963194e+01   84.331613  
std    2.861275e-07  4.785360e-08  9.570720e-08  1.435608e-07  104.736409  
min    9.901466e+01 

...


Output DataFrame Summary:
        lv.E_act   lv.v_ref   la.E_act  implausibility      ao.P_min  \
count  50.000000  50.000000  50.000000       50.000000  5.000000e+01   
mean    9.132979   9.902876   4.795985       95.898285  9.853282e+01   
std     5.317315   5.437533   3.027693       90.654320  7.871425e-07   
min     0.497589   0.816885   0.465025       27.490181  9.853282e+01   
25%     4.487001   5.389273   2.367905       27.490181  9.853282e+01   
50%     8.524874  10.414683   4.788947       27.490181  9.853282e+01   
75%    13.436934  13.783470   6.832400      167.528167  9.853282e+01   
max    18.747681  19.860682  10.408191      287.373166  9.853283e+01   

           ao.P_max      ao.P_pc1      ao.P_pc2      ao.P_pc3          CO  
count  5.000000e+01  5.000000e+01  5.000000e+01  5.000000e+01   50.000000  
mean   9.901466e+01  9.877315e+00  1.975463e+01  2.963194e+01   89.588246  
std    3.163094e-07  5.310055e-08  1.062011e-07  1.593017e-07  107.889060  
min    9.901466e+01 

100%|██████████| 50/50 [00:02<00:00, 22.60it/s]



Output DataFrame Summary:
        lv.E_act   lv.v_ref   la.E_act  implausibility      ao.P_min  \
count  50.000000  50.000000  50.000000       50.000000  5.000000e+01   
mean   10.359544   9.284133   4.784464       88.192088  9.853282e+01   
std     5.406666   5.484608   3.126228       90.387139  8.633285e-07   
min     0.194032   0.119796   0.037257       27.490180  9.853282e+01   
25%     5.132961   5.422089   1.981733       27.490181  9.853282e+01   
50%    11.857791   9.316576   4.703831       27.490181  9.853282e+01   
75%    14.675661  14.392492   7.083468      161.483026  9.853282e+01   
max    19.937433  19.827405  10.264813      303.231545  9.853283e+01   

           ao.P_max      ao.P_pc1      ao.P_pc2      ao.P_pc3          CO  
count  5.000000e+01  5.000000e+01  5.000000e+01  5.000000e+01   50.000000  
mean   9.901466e+01  9.877315e+00  1.975463e+01  2.963194e+01   80.070838  
std    3.644968e-07  6.102525e-08  1.220505e-07  1.830757e-07  107.291680  
min    9.901466e+01 

 36%|███▌      | 18/50 [00:00<00:01, 21.88it/s]
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10712d7f0>>
Traceback (most recent call last):
  File "/Users/mfamili/work/env_ae/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from ModularCirc.Models.NaghaviModel import NaghaviModel, NaghaviModelParameters
from ModularCirc.Analysis.BaseAnalysis import BaseAnalysis
from ModularCirc.Solver import Solver
import pandas as pd
import os
import joblib
from tqdm import tqdm
from typing import Dict, Tuple, List
from autoemulate.history_matching import HistoryMatching


class NaghaviHistoryMatcher:
    def __init__(self, observed_data: Dict[str, np.ndarray], 
                 param_ranges: Dict[str, Tuple[float, float]],
                 n_cycles: int = 40, dt: float = 0.001,
                 implausibility_threshold: float = 3.0,
                 model_discrepancy: float = 0.0):
        """
        Initialize the history matching framework for Naghavi model
        
        Args:
            observed_data: Dictionary containing observed signals (keys: signal names, values: arrays)
            param_ranges: Dictionary of parameter ranges to explore (keys: parameter names, values: (min, max))
            n_cycles: Number of cardiac cycles to simulate
            dt: Time step for simulation
            implausibility_threshold: Threshold for considering points implausible
            model_discrepancy: Model discrepancy term
        """
        self.observed_data = observed_data
        self.param_ranges = param_ranges
        self.param_names = list(param_ranges.keys())
        self.n_cycles = n_cycles
        self.dt = dt
        self.samples = None
        self.simulator_outputs = None
        self.gp_emulators = {}
        
        # Initialize the HistoryMatching object
        self.history_matcher = HistoryMatching(
            threshold=implausibility_threshold,
            discrepancy=model_discrepancy,
            rank=1
        )
        
        # Time setup dictionary
        self.time_setup = {
            "name": "HistoryMatching",
            "ncycles": n_cycles,
            "tcycle": 1.0,  # Will be adjusted per simulation
            "dt": dt,
            "export_min": 1
        }
        # Initialize emulator
        self.emulator = None
        self.emulator_trained = False

    def train_emulator(self, X: np.ndarray, y: np.ndarray):
        """Train the GP emulator using autoemulate"""
        from autoemulate.compare import AutoEmulate
        
        print("Training GP emulator...")
        em = AutoEmulate()
        em.setup(X, y, models=["gp"])
        self.emulator = em.compare()
        self.emulator_trained = True
        print("GP emulator trained successfully")
        
    def get_emulator_predictions(self, X: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Get predictions from trained emulator"""
        if not self.emulator_trained:
            raise ValueError("Emulator not trained yet")
            
        pred_mean, pred_std = self.emulator.predict(X, return_std=True)
        pred_var = np.square(pred_std)
        return pred_mean, pred_var

    def signal_get_pulse(self, signal: np.ndarray, dt: float, num: int = 100) -> Tuple[float, np.ndarray]:
        """
        Resample raw signal to standard resolution
        
        Args:
            signal: Raw signal array
            dt: Original time resolution
            num: Number of points in resampled signal
            
        Returns:
            Tuple of (new_dt, resampled_signal)
        """
        ind = np.argmin(signal)        
        ncycle = len(signal)
        new_signal = np.interp(np.linspace(0, ncycle, num), 
                              np.arange(ncycle), 
                              np.roll(signal, -ind))
        new_dt = ncycle / (num - 1) * dt
        return new_dt, new_signal
    
    def run_simulation(self, params: Dict[str, float]) -> Dict[str, float]:
        """
        Run a single Naghavi model simulation with given parameters
        
        Args:
            params: Dictionary of parameter values
            
        Returns:
            Dictionary of processed features from simulation outputs
        """
        # Create parameter object
        parobj = NaghaviModelParameters()
        
        # Set parameters from input
        for param_name, value in params.items():
            if param_name == "T":
                continue  # Handle cycle time separately
            try:
                obj, param = param_name.split('.')
                parobj._set_comp(obj, [obj], **{param: value})
            except Exception as e:
                print(f"Error setting parameter {param_name}: {e}")
                return None
        
        # Adjust cycle time if specified
        t_cycle = params.get("T", 1.0)
        self.time_setup["tcycle"] = t_cycle
        
        # Create and run model
        try:
            model = NaghaviModel(time_setup_dict=self.time_setup, parobj=parobj, suppress_printing=True)
            solver = Solver(model=model)
            solver.setup(suppress_output=True, optimize_secondary_sv=False, conv_cols=["p_ao"], method='LSODA')
            solver.solve()
            
            if not solver.converged:
                return None
        except Exception as e:
            print(f"Simulation error: {e}")
            return None
        
        # Process results
        raw_results = {}
        
        # Get indices for the last cycle
        tind_fin = np.arange(start=model.time_object.n_t-model.time_object.n_c,
                            stop=model.time_object.n_t)
        
        # Extract raw signals of interest - only min and max values
        for signal_name in self.observed_data.keys():
            try:
                component, _ = signal_name.split('.')
                # Get the pressure signal from the component
                if component in model.components:
                    # Extract P_i (pressure) from the component for the last cycle
                    raw_signal = model.components[component].P_i.values[tind_fin]
                    
                    if '_min' in signal_name:
                        raw_results[signal_name] = np.min(raw_signal)
                    elif '_max' in signal_name:
                        raw_results[signal_name] = np.max(raw_signal)
                else:
                    print(f"Component {component} not found in model")
                    return None
            except Exception as e:
                print(f"Error extracting signal {signal_name}: {e}")
                return None
                
        return raw_results  # Return the processed results directly

    def process_simulation_output(self, raw_results: Dict[str, np.ndarray]) -> Dict[str, float]:
        """
        Process raw simulation outputs to extract features for comparison with observed data
        
        Args:
            raw_results: Dictionary of raw simulation outputs
            
        Returns:
            Dictionary of processed features
        """
        processed = {}
        
        # Process each signal based on what features we want to extract
        for signal_name, signal_data in raw_results.items():
            if '.P' in signal_name:
                # For pressure signals, extract min and max
                if '_min' in signal_name:
                    processed[f"{signal_name}"] = np.min(signal_data)
                elif '_max' in signal_name:
                    processed[f"{signal_name}"] = np.max(signal_data)
            
        return processed
    
    def run_wave(self, samples: pd.DataFrame, n_jobs: int = 1) -> Tuple[pd.DataFrame, List[Dict], np.ndarray]:
        """
        Run a wave of simulations
        
        Args:
            samples: DataFrame of parameter samples
            n_jobs: Number of parallel jobs
            
        Returns:
            Tuple of (successful_samples, outputs, implausibility_scores)
        """
        successful_samples = []
        simulation_outputs = []
        implausibility_scores = []

        # Serial execution
        for _, row in tqdm(samples.iterrows(), total=len(samples)):
            result = self._run_single_sample(row)
            if result is not None:
                params, outputs, impl = result
                successful_samples.append(params)
                simulation_outputs.append(outputs)
                implausibility_scores.append(impl)
                        
        # Create a dataframe with all outputs for inspection
        if successful_samples:
            # Combine parameters and outputs into one dataframe for inspection
            output_df = pd.DataFrame(successful_samples)
            
            # Add implausibility scores
            output_df['implausibility'] = implausibility_scores
            
            # Add simulation outputs for each observed signal
            for i, output_dict in enumerate(simulation_outputs):
                for signal_name, value in output_dict.items():
                    if signal_name not in output_df.columns:
                        output_df[signal_name] = np.nan
                    output_df.at[i, signal_name] = value
            
            # Print summary statistics of the outputs
            print("\nOutput DataFrame Summary:")
            print(output_df.describe())
            
            # Show the relationship between parameters and implausibility
            print("\nCorrelation between parameters and implausibility:")
            for param in self.param_names:
                corr = np.corrcoef(output_df[param], output_df['implausibility'])[0, 1]
                print(f"{param}: {corr:.4f}")
            
            # Show the best match in this wave
            best_idx = np.argmin(implausibility_scores)
            print("\nBest match in this wave:")
            print(output_df.iloc[[best_idx]])
            
            # Print the observed vs. simulated values for the best match
            print("\nObserved vs. Best Simulated Values:")
            for signal_name, observed_value in self.observed_data.items():
                if signal_name in output_df.columns:
                    simulated = output_df.iloc[best_idx][signal_name]
                    print(f"{signal_name}: Observed={observed_value:.4f}, Simulated={simulated:.4f}, Difference={abs(observed_value - simulated):.4f}")
            
            # Save the output dataframe to a CSV file for external analysis
            output_df.to_csv(f"wave_outputs.csv", index=False)
            print("\nOutput dataframe saved to 'wave_outputs.csv' for further analysis")
            
        return pd.DataFrame(successful_samples), simulation_outputs, np.array(implausibility_scores)
    
    
    def _run_single_sample(self, sample: pd.Series) -> Tuple[Dict, Dict, float]:
        """
        Run a single sample and calculate implausibility using emulator if available
        """
        params = sample.to_dict()
        outputs = self.run_simulation(params)
        
        if outputs is None:
            return None
            
        # Prepare predictions and observations for history matching
        predictions = []
        observations = []
        
        for signal_name, obs_value in self.observed_data.items():
            if signal_name not in outputs:
                print(f"Warning: {signal_name} not found in simulation outputs")
                continue
                
            # For direct simulation (no emulator)
            pred_mean = outputs[signal_name]
            pred_var = 0.01  # Small fixed variance for direct simulation
            
            # For emulator, we would use:
            # pred_mean, pred_var = self.get_emulator_predictions(...)
            
            predictions.append([pred_mean, pred_var])
            observations.append([obs_value, 0.1])  # obs_value and obs_variance
            
        if not predictions:
            return None
            
        # Calculate implausibility using HistoryMatching
        try:
            result = self.history_matcher.history_matching(observations, predictions)
            impl = result["I"][0]  # Take the first implausibility value
            return params, outputs, impl
        except Exception as e:
            print(f"Error calculating implausibility: {e}")
            return None
            
    def history_matching(self, n_waves: int = 3, n_samples_per_wave: int = 100, n_jobs: int = 4):
        """
        Perform iterative history matching using the HistoryMatching class
        
        Args:
            n_waves: Number of waves to perform
            n_samples_per_wave: Samples per wave
            n_jobs: Number of parallel jobs
        """
        current_samples = self.generate_samples(n_samples_per_wave)
        all_samples = pd.DataFrame()
        all_implausibilities = np.array([])
        
        for wave in range(n_waves):
            print(f"\nStarting wave {wave + 1}/{n_waves}")
            
            # Run simulations
            successful_samples, outputs, impl_scores = self.run_wave(current_samples, n_jobs)
            print(f"Wave {wave+1}: {len(successful_samples)} successful samples out of {len(current_samples)}")
            
            if len(successful_samples) == 0:
                print("No successful simulations in this wave, generating new random samples")
                current_samples = self.generate_samples(n_samples_per_wave)
                continue  # Try again with new samples instead of stopping
                
            # Store all results
            all_samples = pd.concat([all_samples, successful_samples])
            all_implausibilities = np.concatenate([all_implausibilities, impl_scores]) if len(all_implausibilities) > 0 else impl_scores
            
            # Identify not implausible points (NROY) using threshold from HistoryMatching
            nroy_indices = np.where(impl_scores <= self.history_matcher.threshold)[0]
            not_implausible = successful_samples.iloc[nroy_indices] if len(nroy_indices) > 0 else pd.DataFrame()
            
            print(f"Wave {wave + 1}: {len(not_implausible)} not implausible points found (threshold: {self.history_matcher.threshold})")
            
            # Prepare next wave samples
            if wave < n_waves - 1:
                if len(not_implausible) >= 2:  # Need at least 2 points for meaningful bounds
                    try:
                        print("Generating new samples based on not implausible points")
                        new_sample_points = self.history_matcher._sample_new_points(
                            not_implausible[self.param_names].values, n_samples_per_wave)
                        current_samples = pd.DataFrame(new_sample_points, columns=self.param_names)
                    except Exception as e:
                        print(f"Error generating new samples: {e}")
                        print("Falling back to random sampling")
                        current_samples = self.generate_samples(n_samples_per_wave)
                else:
                    # Fall back to random sampling if too few NROY points
                    print("Too few not implausible points, generating new random samples")
                    current_samples = self.generate_samples(n_samples_per_wave)
                
                print(f"Generated {len(current_samples)} new samples for wave {wave+2}")
            
            # Plot results only if we have implausible and not implausible points
            if len(not_implausible) > 0 and len(successful_samples) > len(not_implausible):
                self.plot_wave_results(wave, successful_samples, impl_scores, not_implausible)
        
        print(f"\nHistory matching completed. Total samples: {len(all_samples)}")
        
        return all_samples, all_implausibilities
    
    def generate_samples(self, n_samples: int, method: str = 'random') -> pd.DataFrame:
        """
        Generate parameter samples using specified method
        
        Args:
            n_samples: Number of samples to generate
            method: Sampling method ('random' or 'lhs')
            
        Returns:
            DataFrame of parameter samples
        """
        samples = []
        
        if method == 'random':
            for param_name, (low, high) in self.param_ranges.items():
                samples.append(np.random.uniform(low, high, n_samples))
        elif method == 'lhs':
            # Implement Latin Hypercube Sampling here if needed
            pass
        else:
            raise ValueError(f"Unknown sampling method: {method}")
            
        return pd.DataFrame(np.array(samples).T, columns=self.param_names)
    
    def plot_wave_results(self, wave: int, samples: pd.DataFrame, 
                         impl_scores: np.ndarray, not_implausible: pd.DataFrame):
        """
        Plot results from a history matching wave
        
        Args:
            wave: Wave number
            samples: All successful samples
            impl_scores: Corresponding implausibility scores
            not_implausible: Not implausible samples
        """
        if len(self.param_names) < 2:
            return
            
        # Select two main parameters to plot
        param1, param2 = self.param_names[:2]
        
        plt.figure(figsize=(12, 5))
        
        # Plot all samples colored by implausibility
        plt.subplot(1, 2, 1)
        sc = plt.scatter(samples[param1], samples[param2], c=impl_scores, 
                        cmap='viridis_r', vmax=3.0)
        plt.colorbar(sc, label='Implausibility')
        plt.xlabel(param1)
        plt.ylabel(param2)
        plt.title(f'Wave {wave + 1} - All Samples')
        
        # Plot not implausible region
        plt.subplot(1, 2, 2)
        plt.scatter(not_implausible[param1], not_implausible[param2], 
                  c='blue', alpha=0.5)
        plt.xlabel(param1)
        plt.ylabel(param2)
        plt.title(f'Wave {wave + 1} - Not Implausible Region')
        
        plt.tight_layout()
        plt.show()

    def calculate_implausibility(self, sim_output: Dict[str, float], 
                            obs_error: Dict[str, float], 
                            model_error: Dict[str, float],
                            emulator_error: Dict[str, float] = None) -> float:
        """
        Calculate combined implausibility metric across all outputs
        
        Args:
            sim_output: Simulator outputs (max values)
            obs_error: Observation errors for each output
            model_error: Model errors for each output
            emulator_error: Optional emulator errors
            
        Returns:
            Combined implausibility score
        """
        total_implausibility = 0.0
        
        for signal_name, obs_value in self.observed_data.items():
            sim_value = sim_output[signal_name]
            error = obs_error.get(signal_name, 0.1)
            m_error = model_error.get(signal_name, 0.05)
            
            # Add emulator error if provided
            e_error = emulator_error.get(signal_name, 0.0) if emulator_error else 0.0
            
            # Calculate absolute difference
            diff = np.abs(sim_value - obs_value)
            impl = diff / np.sqrt(error**2 + m_error**2 + e_error**2)
            total_implausibility += impl
                
        return total_implausibility

    def calculate_implausibility(self, sim_output: Dict[str, float], 
                                obs_error: Dict[str, float], 
                                model_error: Dict[str, float],
                                emulator_error: Dict[str, float] = None) -> float:
        """
        Calculate combined implausibility metric across all processed outputs
        
        Args:
            sim_output: Simulator processed outputs
            obs_error: Observation errors for each output
            model_error: Model errors for each output
            emulator_error: Optional emulator errors
            
        Returns:
            Combined implausibility score
        """
        implausibilities = []
        
        for signal_name, obs_value in self.observed_data.items():
            if signal_name not in sim_output:
                print(f"Warning: {signal_name} not in simulation output")
                continue
                
            sim_value = sim_output[signal_name]
            error = obs_error.get(signal_name, 0.1)
            m_error = model_error.get(signal_name, 0.05)
            
            # Add emulator error if provided
            e_error = emulator_error.get(signal_name, 0.0) if emulator_error else 0.0
            
            # Calculate absolute difference
            diff = np.abs(sim_value - obs_value)
            impl = diff / np.sqrt(error**2 + m_error**2 + e_error**2)
            implausibilities.append(impl)
                
        # Return maximum implausibility (conservative approach)
        # Alternative: return np.mean(implausibilities) for average implausibility
        return np.max(implausibilities)

# Example usage
if __name__ == "__main__":
    # Define observed data (would normally load from file)
    observed_signals = {
        "ao.P_min": 80.0,    # Minimum observed aortic pressure (mmHg)
        "ao.P_max": 120.0,   # Maximum observed aortic pressure (mmHg)
    }

    # Updated parameter ranges
    param_ranges = {
        "lv.E_act": (0.0, 20.0),  # LV active elastance (similar to end-systolic elastance)
        "lv.v_ref": (0.0, 20.0),  # LV reference volume (similar to dead volume)
        "la.E_act": (0., 10.5),   # LA active elastance
        # For systemic arterial resistance, we need to check what parameter name to use
    }

    
    # Create history matcher instance
    hm = NaghaviHistoryMatcher(observed_data=observed_signals, param_ranges=param_ranges)
    
    # Run history matching
    best_samples, best_impl_scores = hm.history_matching(n_waves=5, n_samples_per_wave=50)
    
    # Print best match
    best_idx = np.argmin(best_impl_scores)
    print("\nBest matching parameters:")
    print(best_samples.iloc[best_idx])
    print(f"Implausibility: {best_impl_scores[best_idx]}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Dict, Tuple, List
from autoemulate.compare import AutoEmulate
from autoemulate.history_matching import HistoryMatching
from ModularCirc.Models.NaghaviModel import NaghaviModel, NaghaviModelParameters
from ModularCirc.Solver import Solver

class NaghaviHistoryMatcher:
    def __init__(self, observed_data: Dict[str, float], 
                 param_ranges: Dict[str, Tuple[float, float]],
                 n_cycles: int = 40, dt: float = 0.001,
                 implausibility_threshold: float = 3.0,
                 model_discrepancy: float = 0.0):
        """
        Initialize the history matching framework for Naghavi model
        
        Args:
            observed_data: Dictionary of target values {signal_name: target_value}
            param_ranges: Dictionary of parameter ranges {param_name: (min, max)}
            n_cycles: Number of cardiac cycles to simulate
            dt: Time step for simulation
            implausibility_threshold: Threshold for implausibility
            model_discrepancy: Model discrepancy term
        """
        self.observed_data = observed_data
        self.param_ranges = param_ranges
        self.param_names = list(param_ranges.keys())
        self.n_cycles = n_cycles
        self.dt = dt
        
        # Time setup dictionary
        self.time_setup = {
            "name": "HistoryMatching",
            "ncycles": n_cycles,
            "tcycle": 1.0,
            "dt": dt,
            "export_min": 1
        }
        
        # Initialize emulator
        self.emulator = None
        self.emulator_trained = False
        
        # History matching setup
        self.history_matcher = HistoryMatching(
            threshold=implausibility_threshold,
            discrepancy=model_discrepancy,
            rank=1
        )

    def run_simulation(self, params: Dict[str, float]) -> Dict[str, float]:
        """
        Run a single Naghavi model simulation with given parameters
        Returns dictionary of output values {signal_name: value}
        """
        # Create parameter object
        parobj = NaghaviModelParameters()
        
        # Set parameters from input
        for param_name, value in params.items():
            if param_name == "T":
                continue  # Handle cycle time separately
            try:
                obj, param = param_name.split('.')
                parobj._set_comp(obj, [obj], **{param: value})
            except Exception as e:
                print(f"Error setting parameter {param_name}: {e}")
                return None
        
        # Adjust cycle time if specified
        t_cycle = params.get("T", 1.0)
        self.time_setup["tcycle"] = t_cycle
        
        # Create and run model
        try:
            model = NaghaviModel(time_setup_dict=self.time_setup, parobj=parobj, suppress_printing=True)
            solver = Solver(model=model)
            solver.setup(suppress_output=True, optimize_secondary_sv=False, conv_cols=["p_ao"], method='LSODA')
            solver.solve()
            
            if not solver.converged:
                return None
        except Exception as e:
            print(f"Simulation error: {e}")
            return None
        
        # Process results - extract last cycle min/max values
        raw_results = {}
        tind_fin = np.arange(start=model.time_object.n_t-model.time_object.n_c,
                            stop=model.time_object.n_t)
        
        for signal_name in self.observed_data.keys():
            try:
                component, _ = signal_name.split('.')
                if component in model.components:
                    raw_signal = model.components[component].P_i.values[tind_fin]
                    
                    if '_min' in signal_name:
                        raw_results[signal_name] = np.min(raw_signal)
                    elif '_max' in signal_name:
                        raw_results[signal_name] = np.max(raw_signal)
                else:
                    print(f"Component {component} not found in model")
                    return None
            except Exception as e:
                print(f"Error extracting signal {signal_name}: {e}")
                return None
                
        return raw_results

    def train_emulator(self, X: np.ndarray, y: np.ndarray):
        """Train the GP emulator using autoemulate"""
        print("Training GP emulator...")
        em = AutoEmulate()
        em.setup(X, y, models=["gp"])
        self.emulator = em.compare()
        self.emulator_trained = True
        print(f"GP emulator trained on {len(X)} samples")

    def get_emulator_predictions(self, X: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Get predictions from trained emulator
        Returns:
            pred_mean: Array of shape (n_samples, n_outputs)
            pred_var: Array of shape (n_samples, n_outputs)
        """
        if not self.emulator_trained:
            raise ValueError("Emulator not trained yet")
            
        pred_mean, pred_std = self.emulator.predict(X, return_std=True)
        pred_var = np.square(pred_std)
        return pred_mean, pred_var

    def generate_samples(self, n_samples: int) -> pd.DataFrame:
        """Generate random parameter samples"""
        samples = []
        for param_name, (low, high) in self.param_ranges.items():
            samples.append(np.random.uniform(low, high, n_samples))
        return pd.DataFrame(np.array(samples).T, columns=self.param_names)

    def run_wave(self, samples: pd.DataFrame, use_emulator: bool = False) -> Tuple[pd.DataFrame, np.ndarray]:
        """
        Run a wave of simulations or emulator predictions
        Returns:
            successful_samples: DataFrame of parameter sets
            impl_scores: Array of implausibility scores
        """
        if use_emulator and self.emulator_trained:
            # Use emulator for predictions
            X = samples[self.param_names].values
            pred_mean, pred_var = self.get_emulator_predictions(X)
            
            # Calculate implausibility for each sample
            impl_scores = []
            for i in range(len(samples)):
                predictions = [[pred_mean[i,j], pred_var[i,j]] 
                              for j in range(len(self.observed_data))]
                observations = [[obs, 0.1] for obs in self.observed_data.values()]
                
                result = self.history_matcher.history_matching(observations, predictions)
                impl_scores.append(result["I"][0])
                
            return samples, np.array(impl_scores)
        else:
            # Run actual simulations
            successful_samples = []
            impl_scores = []
            
            for _, row in tqdm(samples.iterrows(), total=len(samples)):
                params = row.to_dict()
                outputs = self.run_simulation(params)
                
                if outputs is None:
                    continue
                    
                # Prepare predictions and observations
                predictions = [[outputs[signal], 0.01]  # Small fixed variance
                              for signal in self.observed_data]
                observations = [[obs, 0.1]  # Observation variance
                              for obs in self.observed_data.values()]
                
                result = self.history_matcher.history_matching(observations, predictions)
                impl_scores.append(result["I"][0])
                successful_samples.append(params)
            
            return pd.DataFrame(successful_samples), np.array(impl_scores)

    def history_matching(self, n_waves: int = 3, n_samples_per_wave: int = 100, 
                        use_emulator: bool = False):
        """
        Perform iterative history matching
        Returns:
            all_samples: DataFrame of all evaluated parameter sets
            all_impl_scores: Array of corresponding implausibility scores
        """
        current_samples = self.generate_samples(n_samples_per_wave)
        all_samples = pd.DataFrame()
        all_impl_scores = np.array([])
        
        for wave in range(n_waves):
            print(f"\n=== Wave {wave + 1}/{n_waves} ===")
            
            # Determine if we should use emulator (after first wave if enabled)
            wave_use_emulator = use_emulator and (wave > 0) and self.emulator_trained
            
            # Run the wave
            successful_samples, impl_scores = self.run_wave(
                current_samples, 
                use_emulator=wave_use_emulator
            )
            
            print(f"Evaluated {len(successful_samples)} samples")
            print(f"Min implausibility: {np.min(impl_scores):.2f}")
            print(f"Max implausibility: {np.max(impl_scores):.2f}")
            
            # Store results
            all_samples = pd.concat([all_samples, successful_samples])
            all_impl_scores = np.concatenate([all_impl_scores, impl_scores])
            
            # Identify Not Ruled Out Yet (NROY) points
            nroy_mask = impl_scores <= self.history_matcher.threshold
            nroy_samples = successful_samples[nroy_mask]
            print(f"NROY points: {len(nroy_samples)}")
            
            # Train emulator after first wave if requested
            if wave == 0 and use_emulator and len(successful_samples) > 10:
                X_train = successful_samples[self.param_names].values
                # Collect all outputs for training data
                y_train = []
                for _, row in successful_samples.iterrows():
                    outputs = self.run_simulation(row.to_dict())
                    if outputs is not None:
                        y_train.append([outputs[signal] for signal in self.observed_data])
                y_train = np.array(y_train)
                self.train_emulator(X_train, y_train)
            
            # Generate new samples for next wave
            if wave < n_waves - 1:
                if len(nroy_samples) >= 2:
                    # Sample from NROY space
                    new_samples = self.history_matcher._sample_new_points(
                        nroy_samples[self.param_names].values, 
                        n_samples_per_wave
                    )
                    current_samples = pd.DataFrame(new_samples, columns=self.param_names)
                else:
                    # Fall back to random sampling
                    print("Not enough NROY points - using random sampling")
                    current_samples = self.generate_samples(n_samples_per_wave)
        
        return all_samples, all_impl_scores

    def plot_results(self, samples: pd.DataFrame, impl_scores: np.ndarray):
        """Plot 2D parameter space with implausibility"""
        if len(self.param_names) < 2:
            return
            
        fig, ax = plt.subplots(figsize=(10, 8))
        sc = ax.scatter(samples[self.param_names[0]], 
                        samples[self.param_names[1]], 
                        c=impl_scores,
                        cmap='viridis_r',
                        vmin=0,
                        vmax=self.history_matcher.threshold*1.5)
        
        ax.set_xlabel(self.param_names[0])
        ax.set_ylabel(self.param_names[1])
        ax.set_title("History Matching Results")
        plt.colorbar(sc, label='Implausibility')
        plt.show()


# Example usage
if __name__ == "__main__":
    # Define observed data (target values)
    observed_signals = {
        "ao.P_min": 80.0,    # Minimum aortic pressure (mmHg)
        "ao.P_max": 120.0,   # Maximum aortic pressure (mmHg)
    }

    # Define parameter ranges to explore
    param_ranges = {
        "lv.E_act": (0.0, 20.0),  # LV active elastance
        "lv.v_ref": (0.0, 20.0),  # LV reference volume
        "la.E_act": (0.0, 10.5),  # LA active elastance
    }

    # Create history matcher instance
    hm = NaghaviHistoryMatcher(
        observed_data=observed_signals,
        param_ranges=param_ranges,
        implausibility_threshold=3.0
    )

    # Run history matching (with emulator after first wave)
    all_samples, all_impl_scores = hm.history_matching(
        n_waves=3,
        n_samples_per_wave=50,
        use_emulator=True
    )

    # Plot results
    hm.plot_results(all_samples, all_impl_scores)

    # Print best parameters
    best_idx = np.argmin(all_impl_scores)
    print("\nBest parameters found:")
    print(all_samples.iloc[best_idx])
    print(f"Implausibility: {all_impl_scores[best_idx]:.2f}")

In [ ]:
from typing import Dict, Tuple
import numpy as np
from autoemulate.history_matching import HistoryMatcher, BaseSimulator



if __name__ == "__main__":
    # Define your parameter ranges
    param_ranges = {
        "lv.E_act": (0.0, 20.0),  # LV active elastance
        "lv.v_ref": (0.0, 20.0),  # LV reference volume
        "la.E_act": (0.0, 10.5),  # LA active elastance
    }
    
    # Define observed data with means and variances
    observations = {
        "ao.P_min": (80.0, 0.1),    # (mean, variance) for minimum aortic pressure
        "ao.P_max": (120.0, 0.1),    # (mean, variance) for maximum aortic pressure
    }
    
    # Create simulator instance
    simulator = NaghaviSimulator(
        observed_data={k: v[0] for k, v in observations.items()},  # Extract just the means
        param_ranges=param_ranges,
        n_cycles=40,
        dt=0.001
    )
    
    # Test generating samples
    samples = simulator.generate_initial_samples(10)
    print("Generated samples:", samples)

    # Create history matcher
    hm = HistoryMatcher(
        simulator=simulator,
        observations=observations,  # This needs both means and variances
        threshold=3.0
    )

    # Run history matching
    all_samples, all_impl_scores = hm.run_history_matching(
        n_waves=20,
        n_samples_per_wave=50,
        use_emulator=True
    )